In [ ]:
import pandas as pd
import numpy as np
import os
import time
import warnings
from tqdm.auto import tqdm


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb
from itertools import combinations

from sklearn.metrics import accuracy_score, classification_report
import random
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

from itertools import combinations
import tqdm
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/combined.csv')

In [ ]:
columns_list = df.columns

NameError: name 'df' is not defined

In [ ]:
'range_ycrcb_y_range' in columns_list

True

In [ ]:
# 14 features
basic_cols = ["area", "length", "width", "ratio", "major_axis_length", "minor_axis_length",
              "convex_hull_area", "convex_hull_perimeter", "equivalent_diameter", "aspect_ratio",
              "perimeter", "roundness", "compactness", "solidity","shape_factor_1", "shape_factor_2", "shape_factor_3", "shape_factor_4"]

# 4 features
texture_cols = ["texture_third_moment", "texture_mean", "texture_std", "texture_uniformity"]
# 4 features
# shape_factor_cols = ["shape_factor_1", "shape_factor_2", "shape_factor_3", "shape_factor_4"]
# 63 features
color_rgb_hsv_lab_cols = ["mean_r", "mean_g", "mean_B",
                          "sqrt_r", "sqrt_g", "sqrt_B",
                          "std_r", "std_g", "std_B",
                          "skew_r", "skew_g", "skew_B",
                          "kur_r", "kur_g", "kur_B",


                          "mean_h", "mean_s", "mean_v",
                          "std_h", "std_s", "std_v",
                          "sqrt_h", "sqrt_s", "sqrt_v",
                          "skew_h", "skew_s", "skew_v",
                          "kur_h", "kur_s", "kur_v",


                          "mean_l", "mean_a", "mean_b",
                          "std_l", "std_a", "std_b",
                          "sqrt_l", "sqrt_a", "sqrt_b",
                          "skew_l", "skew_a", "skew_b",
                          "kur_l", "kur_a", "kur_b"]


# 42 features
color_ycrcb_xyz_cols = ["mean_y", "mean_cb", "mean_cr",
                        "std_y", "std_cb", "std_cr",
                        "sqrt_y", "sqrt_cb", "sqrt_cr",
                        "skew_y", "skew_cb", "skew_cr",
                        "kur_y", "kur_cb", "kur_cr",


                        "mean_x", "mean_Y", "mean_z",
                        "std_x", "std_Y", "std_z",
                        "sqrt_x", "sqrt_Y", "sqrt_z",
                        "skew_x", "skew_Y", "skew_z",
                        "kur_x", "kur_Y", "kur_z",]


# 75 features
pf_cols = ['pf_bgr_b_p5', 'pf_bgr_b_p25', 'pf_bgr_b_p50', 'pf_bgr_b_p75', 'pf_bgr_b_p95',
           'pf_bgr_g_p5', 'pf_bgr_g_p25', 'pf_bgr_g_p50', 'pf_bgr_g_p75', 'pf_bgr_g_p95',
           'pf_bgr_r_p5', 'pf_bgr_r_p25', 'pf_bgr_r_p50', 'pf_bgr_r_p75', 'pf_bgr_r_p95',

           'pf_hsv_h_p5', 'pf_hsv_h_p25', 'pf_hsv_h_p50', 'pf_hsv_h_p75', 'pf_hsv_h_p95',
           'pf_hsv_s_p5', 'pf_hsv_s_p25', 'pf_hsv_s_p50', 'pf_hsv_s_p75', 'pf_hsv_s_p95',
           'pf_hsv_v_p5', 'pf_hsv_v_p25', 'pf_hsv_v_p50', 'pf_hsv_v_p75', 'pf_hsv_v_p95',

           'pf_lab_l_p5', 'pf_lab_l_p25', 'pf_lab_l_p50', 'pf_lab_l_p75', 'pf_lab_l_p95',
           'pf_lab_a_p5', 'pf_lab_a_p25', 'pf_lab_a_p50', 'pf_lab_a_p75', 'pf_lab_a_p95',
           'pf_lab_b_p5', 'pf_lab_b_p25', 'pf_lab_b_p50', 'pf_lab_b_p75', 'pf_lab_b_p95',

           'pf_ycrcb_y_p5', 'pf_ycrcb_y_p25', 'pf_ycrcb_y_p50', 'pf_ycrcb_y_p75', 'pf_ycrcb_y_p95',
           'pf_ycrcb_cr_p5', 'pf_ycrcb_cr_p25', 'pf_ycrcb_cr_p50', 'pf_ycrcb_cr_p75', 'pf_ycrcb_cr_p95',
           'pf_ycrcb_cb_p5', 'pf_ycrcb_cb_p25', 'pf_ycrcb_cb_p50', 'pf_ycrcb_cb_p75', 'pf_ycrcb_cb_p95',

           'pf_xyz_x_p5', 'pf_xyz_x_p25', 'pf_xyz_x_p50', 'pf_xyz_x_p75', 'pf_xyz_x_p95',
           'pf_xyz_y_p5', 'pf_xyz_y_p25', 'pf_xyz_y_p50', 'pf_xyz_y_p75', 'pf_xyz_y_p95',
           'pf_xyz_z_p5', 'pf_xyz_z_p25', 'pf_xyz_z_p50', 'pf_xyz_z_p75', 'pf_xyz_z_p95'
           ]

# 9 features
var_ratio_cols = ['var_ratio_R_G', 'var_ratio_R_B', 'var_ratio_G_B',
                  'var_ratio_H_S', 'var_ratio_H_V', 'var_ratio_S_V',
                  'var_ratio_L_A', 'var_ratio_L_B', 'var_ratio_A_B']

# 60 features
range_cols = ['range_bgr_b_range', 'iqr_bgr_b_iqr', 'mv_bgr_b_mode', 'mc_bgr_b_mode_conc',
              'range_bgr_g_range', 'iqr_bgr_g_iqr', 'mv_bgr_g_mode', 'mc_bgr_g_mode_conc',
              'range_bgr_r_range', 'iqr_bgr_r_iqr', 'mv_bgr_r_mode', 'mc_bgr_r_mode_conc',

              'range_hsv_h_range', 'iqr_hsv_h_iqr', 'mv_hsv_h_mode', 'mc_hsv_h_mode_conc',
              'range_hsv_s_range', 'iqr_hsv_s_iqr', 'mv_hsv_s_mode', 'mc_hsv_s_mode_conc',
              'range_hsv_v_range', 'iqr_hsv_v_iqr', 'mv_hsv_v_mode', 'mc_hsv_v_mode_conc',

              'range_lab_l_range', 'iqr_lab_l_iqr', 'mv_lab_l_mode', 'mc_lab_l_mode_conc',
              'range_lab_a_range', 'iqr_lab_a_iqr', 'mv_lab_a_mode', 'mc_lab_a_mode_conc',
              'range_lab_b_range', 'iqr_lab_b_iqr', 'mv_lab_b_mode', 'mc_lab_b_mode_conc',

              'range_ycrcb_y_range', 'iqr_ycrcb_y_iqr', 'mv_ycrcb_y_mode', 'mc_ycrcb_y_mode_conc',
              'range_ycrcb_cr_range', 'iqr_ycrcb_cr_iqr', 'mv_ycrcb_cr_mode', 'mc_ycrcb_cr_mode_conc',
              'range_ycrcb_cb_range', 'iqr_ycrcb_cb_iqr', 'mv_ycrcb_cb_mode', 'mc_ycrcb_cb_mode_conc',

              'range_xyz_x_range', 'iqr_xyz_x_iqr', 'mv_xyz_x_mode', 'mc_xyz_x_mode_conc',
              'range_xyz_y_range', 'iqr_xyz_y_iqr', 'mv_xyz_y_mode', 'mc_xyz_y_mode_conc',
              'range_xyz_z_range', 'iqr_xyz_z_iqr', 'mv_xyz_z_mode', 'mc_xyz_z_mode_conc'
              ]

# 10 features
lbp_cols = [f"LBP_{i}" for i in range(10)]
# 16 features
glcm_cols = [f"GLCM_{i}" for i in range(16)]
# 25 features
zernike_cols = [f"Zernike_{i}" for i in range(25)]


all_features = {"Basic": basic_cols,
                "Texture": texture_cols,
                "Color_RGB_HSV_Lab": color_rgb_hsv_lab_cols,
                # "Color_YCRCB_XYZ": color_ycrcb_xyz_cols,
                "LBP": lbp_cols,
                "GLCM": glcm_cols,
                "Zernike": zernike_cols,
                "PF": pf_cols,
                "Var_Ratio": var_ratio_cols,
                "Range": range_cols}



In [ ]:

feature_combinations = []
for r in range(5, len(all_features) + 1):
    for combo in combinations(all_features.keys(), r):
        feature_combinations.append(combo)


In [ ]:

feature_combinations = []
for r in range(2, 5):
    for combo in combinations(all_features.keys(), r):
        feature_combinations.append(combo)


In [ ]:
for r in all_features.keys():
    feature_combinations.append((r))

In [ ]:
models = {
    # "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42),
    # "SVM": SVC(kernel='rbf', random_state=42),
    # "Random Forest": RandomForestClassifier(n_estimators=89, random_state=42),
    # "Gradient Boosting": GradientBoostingClassifier(n_estimators=75, random_state=42),
    # "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=8),
    "LightGBM": lgb.LGBMClassifier(n_estimators=88, random_state=42, verbose=-1)
}

In [ ]:
features = ['Basic',
 'Texture',
 'Color_RGB_HSV_Lab',
 'LBP',
 'GLCM',
 'Zernike',
 'PF',
 'Var_Ratio',
 'Range']

### Single feature

In [ ]:
all_results = []


for single in features:

    df = pd.read_csv('/content/drive/MyDrive/combined.csv')
    df_selected = df[all_features[f'{single}']+['label']]
    X = df_selected.drop(columns=['label'])
    y = df_selected['label']

    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    for name, model in tqdm.tqdm(models.items(), 'Training models..'):
        # print(f"🚀 Training {name} on {feature_combo}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

        all_results.append({
            "Model": name,
            "Features combination": single,
            "Num Features": len(single),
            "Accuracy": round(accuracy, 4),
            "Precision": round(precision, 4),
            "Recall": round(recall, 4),
            "F1 Score": round(f1, 4),
            "CV_scores(5fold)": round(np.mean(cv_scores), 4)
        })

        results_df = pd.DataFrame(all_results)

        output_path = '/content/drive/MyDrive/model_results_knn.csv'
        if os.path.exists(output_path):
            results_df.to_csv(output_path, mode='a', header=False, index=False)
        else:
            results_df.to_csv(output_path, mode='w', header=True, index=False)

# results_df = pd.DataFrame(all_results)

Training models..: 100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


### Combine features

In [ ]:
all_results = []

for feature_combo in tqdm.tqdm(feature_combinations[16::], 'Choosing combinations of features..'):
    selected_features = []

    for group in feature_combo:
        selected_features.extend(all_features[group])

    df = pd.read_csv('/content/drive/MyDrive/combined.csv')
    df_selected = df[selected_features+['label']]
    X = df_selected.drop(columns=['label'])
    y = df_selected['label']

    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    for name, model in tqdm.tqdm(models.items(), 'Training models..'):
        # print(f"🚀 Training {name} on {feature_combo}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

        all_results.append({
            "Model": name,
            "Features combination": feature_combo,
            "Num Features": len(selected_features),
            "Accuracy": round(accuracy, 4),
            "Precision": round(precision, 4),
            "Recall": round(recall, 4),
            "F1 Score": round(f1, 4),
            "CV_scores(5fold)": round(np.mean(cv_scores), 4)
        })

        results_df = pd.DataFrame(all_results)

        output_path = '/content/drive/MyDrive/model_results_lgbm_rev.csv'
        if os.path.exists(output_path):
            results_df.to_csv(output_path, mode='a', header=False, index=False)
        else:
            results_df.to_csv(output_path, mode='w', header=True, index=False)

results_df = pd.DataFrame(all_results)

Choosing combinations of features..:  27%|██▋       | 65/240 [4:43:58<12:44:33, 262.14s/it]


KeyboardInterrupt: 

In [ ]:
results_df.to_csv('svm_model.csv',index=False)